In [1]:
import pandas as pd

import numpy as np

import re

In [2]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_algo_names = ['BinarySearch', 'BubbleSort', 'Factorial', 'Fibonacci', 'IntegerBinary', 'MultiplyMatrix', 'PrimeFactors', 'ReverseString']

config_id_variable = "CASE"
config_uid_variable = "ST17_01"

config_answer_variables = [
    ['TR21_01', 'BR21_01', 'TI21_01', 'BI21_01'],
    ['TR22_01', 'BR22_01', 'TI22_01', 'BI22_01'],
    ['TR23_01', 'BR23_01', 'TI23_01', 'BI23_01'],
    ['TR24_01', 'BR24_01', 'TI24_01', 'BI24_01'],
    ['TR25_01', 'BR25_01', 'TI25_01', 'BI25_01'],
    ['TR26_01', 'BR26_01', 'TI26_01', 'BI26_01'],
    ['TR27_01', 'BR27_01', 'TI27_01', 'BI27_01'],
    ['TR28_01', 'BR28_01', 'TI28_01', 'BI28_01'],
]

config_answer_patterns = [
    ['.*3.*', '.*3.*', '.*3.*', '.*3.*'],
    ['.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*'],
    ['.*6.*','.*6.*','.*120.*','.*120.*'],
    ['.*2.*', '.*2.*','.*2.*','.*2.*'],
    ['.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*'],
    ['.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*'],
    ['.*3.*5.*','.*3.*5.*','.*3.*5.*','.*3.*5.*'],
    ['.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*']
]

#config_names =  'ST17_01'

config_datasheet = r'./BookGD.xlsx'

In [3]:
print(config_datasheet)

./BookGD.xlsx


In [4]:
df_matrix = []
raw = pd.read_excel(config_datasheet)

#print(df)
# load data
for row, _stuff in enumerate(config_algo_names):
    df_array = []
    for col, _stuff in enumerate(config_prefix):
        df = pd.DataFrame(raw, columns = [config_id_variable,config_uid_variable,
                                          config_answer_variables[row][col]])
        df = df.iloc[1:]
        df = df.dropna()
        df_array.append(df)
        
        
    df_matrix.append(df_array)
    #print(df_matrix)

In [5]:
#print(df_matrix)
cols = ["id", "uid","correctness", "count_val", "algo_name"]
df_BU_R = pd.DataFrame(columns=cols)
#print(df_BU_R)
df_BU_I = pd.DataFrame(columns=cols)
df_TD_R = pd.DataFrame(columns=cols)
df_TD_I = pd.DataFrame(columns=cols)

for row_idx, df_row in enumerate(df_matrix):
    algo_name = config_algo_names[row_idx]
    for col, df in enumerate(df_row): 
        pattern = config_answer_patterns[row_idx][col]
        regex = re.compile(pattern)
        
        for index, row in df.iterrows():

            id_value =  row["CASE"]
            uid_value =row["ST17_01"]
            
            answer = row[config_answer_variables[row_idx][col]]
          
            result = regex.match(str(answer))
            correctness = False
            count = 0
            
            if result is not None:
                correctness = True
                count += 1
                
            flag = ""
            #print(flag)
            data = pd.DataFrame([[id_value,uid_value, correctness, count, algo_name]], columns=cols)
                  
            if col == 0:
                df_BU_R = df_BU_R.append(data)
                
            elif col == 1:
                df_BU_I = df_BU_I.append(data)
            elif col == 2:
                df_TD_R = df_TD_R.append(data)
            else:
                df_TD_I = df_TD_I.append(data)

In [6]:
df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]

In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
cols = ["Subject","MNumber","ProgrammingStyle", "Comprehension", "Algorithm","count_val", "Correctness"]

def fill(list_data, dataframe, coding_style, comprehension_style):
    for idx, row in dataframe.iterrows():
    
        algorithm = row["algo_name"]
        count = row["count_val"]
        
        correctness = row["correctness"]
        id = row["id"]
        uid = row["uid"]
        list_data.append([id,uid,coding_style, comprehension_style, algorithm, count, correctness])

data = []
fill(data, df_BU_R, "R", "BU")
fill(data, df_BU_I, "I", "BU")
fill(data, df_TD_R, "R", "TD")
fill(data, df_TD_I, "I", "TD")
df = pd.DataFrame(data, columns=cols)

print (df)
df.to_excel("./results/Assignment7.xlsx", index=False)

     Subject MNumber ProgrammingStyle Comprehension      Algorithm  count_val  \
0        125  683252                R            BU   BinarySearch          1   
1        131  718523                R            BU   BinarySearch          1   
2        154  679008                R            BU   BinarySearch          1   
3        172  665346                R            BU   BinarySearch          1   
4        173  677314                R            BU   BinarySearch          1   
..       ...     ...              ...           ...            ...        ...   
846      357  681740                I            TD  ReverseString          1   
847      379  679206                I            TD  ReverseString          1   
848      386  635459                I            TD  ReverseString          0   
849      392  681807                I            TD  ReverseString          0   
850      402  672559                I            TD  ReverseString          1   

     Correctness  
0       

In [13]:

df_matrix = []
config_datasheet1 = r'./results/Assignment7.xlsx'
df = pd.read_excel(config_datasheet1)

grouped = df.groupby('MNumber',)
df = grouped['count_val'].agg(np.sum)
#df_matrix.append(grouped['count_val'].agg(np.sum))
print(df)
#writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

with pd.ExcelWriter('./results/Assignment7.xlsx', engine="openpyxl",
                    mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_3')



MNumber
424000    7
437343    4
455428    7
480237    8
486086    8
         ..
713333    6
713812    7
714231    6
718523    7
718817    4
Name: count_val, Length: 110, dtype: int64


In [ ]:
#print(df.iloc[1]['Subject'])
   
    #if df.iloc[index]['Subject'] is df.iloc[1]['Subject']:
    # print('hey') '
    #for index, row in df.iterrows():
    #if df.loc[df['Subject'] is df.iloc[index]['Subject']:
             # df.iloc[index].count.sum()
  #  else:
             # print("a is greater than b")
    #for index in df.iterrows():
   # value = df['Subject'].loc[2]
    #print(value)
    #test = df.loc[df['Subject'] == value, 'count'].sum()
#print("value is ",test)
#df.groupby('Subject').count.sum()